##1. Load raw csv data as NumPy array for numerical processing
##2. Features: exclude customer ID (column 0) and target (last column)
##3. Target: whether customer bought again (1) or not (0)


In [3]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
raw_csv_data = np.loadtxt('/content/drive/MyDrive/Audiobooks_data (1).csv', delimiter = ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

**Balance the Data Set**


##1. Count number of positive (converted) customers
##2. Remove excess negative samples to balance classes


In [5]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
  if targets_all[i] == 0:
    zero_targets_counter += 1
    if zero_targets_counter > num_one_targets:
      indices_to_remove.append(i)


unscaled_inputs_withequalpriors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equalpriors = np.delete(targets_all, indices_to_remove, axis = 0)

**Standardize the inputs**

---



## Standardize inputs to have mean 0 and variance 1

In [6]:
scaled_inputs = preprocessing.scale(unscaled_inputs_withequalpriors)

**Shuffle the Data**

## Shuffled the data to remove any ordering bias

In [30]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equalpriors[shuffled_indices]

# Split into train (80%), validation (10%), test (10%)
### I checked the number of count of samples into each dataset to ensure that they are equally prior to classes (Two-Classes). They should be almost 50% for each class.

In [31]:
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - validation_samples_count - train_samples_count

# Now Extract the data from the big data
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count: train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count: train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)
print(np.sum(shuffled_targets)/samples_count)

1791.0 3579 0.5004191114836547
225.0 447 0.5033557046979866
221.0 448 0.49330357142857145
0.5


###save the files as .npz file format which is the TensorFlow friendly format

In [32]:
np.savez('AudioBook_data_train', inputs = train_inputs, targets = train_targets)
np.savez('AudioBook_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('AudioBook_data_test', inputs = test_inputs, targets = test_targets)

## Extract inputs and targets from train, test and validation dataset

In [33]:
npz1 = np.load('AudioBook_data_train.npz')

train_inputs = npz1['inputs'].astype(float)
train_targets = npz1['targets'].astype(int)

npz2 = np.load('AudioBook_data_validation.npz')
validation_inputs, validation_targets = npz2['inputs'].astype(float), npz2['targets'].astype(int)

npz3 = np.load('AudioBook_data_test.npz')
test_inputs, test_targets = npz3['inputs'].astype(float), npz3['targets'].astype(int)

In [34]:
import tensorflow as tf

#I selected these while outlining the model,

##1. ReLU → avoids vanishing gradients

##2. Two hidden layers → capture non-linear customer behavior

##3. Softmax → probability distribution over classes

##4. Adam → adaptive learning rate for faster convergence

In [40]:
#  Model

input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## I chose appropriate hyperparameters after few experimentations and fine tuning them.
## Early stopping prevents overfitting and improves generalization.

In [47]:
batch_size = 70
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size = batch_size,
          epochs = max_epochs,
          callbacks = [early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose = 2)

Epoch 1/100
52/52 - 0s - 6ms/step - accuracy: 0.8318 - loss: 0.3055 - val_accuracy: 0.7987 - val_loss: 0.3388
Epoch 2/100
52/52 - 1s - 11ms/step - accuracy: 0.8363 - loss: 0.3062 - val_accuracy: 0.8121 - val_loss: 0.3362
Epoch 3/100
52/52 - 0s - 6ms/step - accuracy: 0.8276 - loss: 0.3112 - val_accuracy: 0.8166 - val_loss: 0.3343
Epoch 4/100
52/52 - 0s - 6ms/step - accuracy: 0.8332 - loss: 0.3057 - val_accuracy: 0.8210 - val_loss: 0.3443
Epoch 5/100
52/52 - 0s - 6ms/step - accuracy: 0.8354 - loss: 0.3032 - val_accuracy: 0.8143 - val_loss: 0.3419


#### Hence I got the validation about 81%. which is almost good.

# Testing our model on the test dataset. which has been unseen to our model while training.

In [48]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8345 - loss: 0.3156 


In [52]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print('Test accuracy: {0: .2f}%,  Test loss: {1: .2f}%'.format(test_accuracy*100, test_loss*100.))

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8345 - loss: 0.3156 
Test accuracy:  83.26%,  Test loss:  33.16%


#Model predicts the targets with around 83% accuracy on real world preprocessed data.
#i.e.  If we are given 10 customers and their audiobook activity we will be able to correctly identify future customer behaviour of 8 of them.